<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf1_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


1.15.2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 1.15.2
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [0]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [0]:
print(np.max(x_train), np.min(x_train))

255 0


In [0]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)

y_train = np.asarray(y_train, dtype = np.int64) #更改numpy的数据类型
y_valid = np.asarray(y_valid, dtype = np.int64)
y_test = np.asarray(y_test, dtype = np.int64)


In [0]:
print(np.max(x_train_scaled), np.min(x_train_scaled))

2.0231433 -0.8105136


In [0]:
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [0]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train,
                       epochs = epochs,
                       batch_size = batch_size)
for data, label in dataset.take(1):
    print(data)
    print(label)

#像tensorflow2.0那样是没法工作的

RuntimeError: ignored

In [31]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train,
                       epochs = epochs,
                       batch_size = batch_size)
# 1. auto initialization
# 2. can't be re-initialized. make_initializable_iterator
dataset_iter = dataset.make_one_shot_iterator() #他可以创建一个遍历器，去遍历这个dataset
#得到这个dataset_iter之后，我们可以通过get_next()函数来得到下一组的数据
#make_one_shot_iterator特点
#1.自动初始化，我们在打开session的时候并没有调用初始化函数，这个dataset_iter就可以直接用了
#2.不能够被重新初始化，这里面我们的dataset是使用x_train_scaled，y_train来组成的
# 如果想用别的数据来重新初始化这个dataset那么是不可以的。
# 而另外一个API make_initializable_iterator则是可以的

x, y = dataset_iter.get_next() #x y还不是具体的值，还是个tensor
with tf.Session() as sess:
  for i in range(5):
    x_val, y_val = sess.run([x, y])
    print(x_val)
    print(y_val)
    

[[-0.8105136  -0.8105136  -0.8105136  ... -0.8105136  -0.8105136
  -0.8105136 ]
 [-0.8105136  -0.8105136  -0.8105136  ... -0.8105136  -0.8105136
  -0.8105136 ]
 [-0.8105136  -0.8105136  -0.8105136  ... -0.8105136  -0.8105136
  -0.8105136 ]
 ...
 [-0.8105136  -0.8105136  -0.8105136  ...  0.20071293 -0.5549289
  -0.8105136 ]
 [-0.8105136  -0.8105136  -0.8105136  ... -0.8105136  -0.8105136
  -0.8105136 ]
 [-0.8105136  -0.8105136  -0.8105136  ... -0.8105136  -0.8105136
  -0.8105136 ]]
[9 1 8 0 5 4 6 7 7 5 2 1 0 7 9 0 0 4 0 2]
[[-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]
 [-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]
 [-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]
 ...
 [-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]
 [-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]
 [-0.8105136 -0.8105136 -0.8105136 ... -0.8105136 -0.8105136 -0.8105136]]
[3 4 9 8 2 8 3 5 8 7 4 4 1 8

In [0]:
hidden_units = [100, 100]
class_num = 10

#这里面x和y已经是对应的数据和label的tensor了
input_for_next_layer = x #=x相当于把我们的dataset接入到我们的计算图中来了
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                           hidden_unit,
                                           activation=tf.nn.relu)
logits = tf.layers.dense(input_for_next_layer,
                         class_num)
# last_hidden_output * W(logits) -> softmax -> prob
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels = y,
                                              logits = logits)
# get accuracy.
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [0]:
print(x) #x变成了Iterator tensor
print(logits)

Tensor("IteratorGetNext_9:0", shape=(?, 784), dtype=float32)
Tensor("dense_5/BiasAdd:0", shape=(?, 10), dtype=float32)


In [0]:
# session

init = tf.global_variables_initializer()
train_steps_per_epoch = x_train.shape[0] // batch_size

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            #x_val, y_val = sess.run([x, y])
            #print(x_val.shape)
            #print(y_val.shape)
           
            loss_val, accuracy_val, _ = sess.run(
                [loss, accuracy, train_op])
            print('\r[Train] epoch: %d, step: %d, loss: %3.5f, accuracy: %2.2f' % (
                epoch, step, loss_val, accuracy_val), end="")

[Train] epoch: 9, step: 2749, loss: 0.60225, accuracy: 0.85